In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Input
from keras.applications import MobileNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_recall_fscore_support, matthews_corrcoef
import cv2
import os
import random
import time
from zipfile import ZipFile
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import gc  # Added for garbage collection
import xgboost as xgb
from itertools import product

# Set random seeds function
def set_seeds(seed_value):
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)

seed_value = 42
set_seeds(seed_value)

# Mount Google Drive and extract data
from google.colab import drive
drive.mount('/content/drive')

# Extract zip file
zip_path = '/content/drive/My Drive/sartaj1.zip'
extract_path = '/content/extracted_data'

try:
    with ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Data extracted successfully!")
except Exception as e:
    print(f"Error extracting data: {e}")

# Find the correct data path
def find_data_path():
    categories = ["glioma", "meningioma", "notumor", "pituitary"]
    for root, dirs, files in os.walk(extract_path):
        if all(category in dirs for category in categories):
            return root
    return extract_path

data_path = find_data_path()
print(f"Data path: {data_path}")

categories = ["glioma", "meningioma", "notumor", "pituitary"]
labels = ["Glioma", "Meningioma", "No Tumor", "Pituitary"]
IMG_SIZE = 224

# Create file paths list
def create_file_paths_list():
    """Create list of file paths and labels WITHOUT loading images"""
    file_paths = []
    file_labels = []

    for category in categories:
        path = os.path.join(data_path, category)
        if not os.path.exists(path):
            print(f"Warning: Path {path} does not exist!")
            continue

        class_num = categories.index(category)
        image_files = [f for f in os.listdir(path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        print(f"Found {len(image_files)} images in {category}")

        for img_file in image_files:
            file_paths.append(os.path.join(path, img_file))
            file_labels.append(class_num)

    print(f"Total files found: {len(file_paths)}")
    print(f"Class distribution: {np.bincount(file_labels)}")

    # Shuffle the data
    combined = list(zip(file_paths, file_labels))
    random.shuffle(combined)
    file_paths, file_labels = zip(*combined)

    return np.array(file_paths), np.array(file_labels)

# Enhanced image loading with better preprocessing
def load_image_batch(file_paths, labels, indices, img_size=224):
    """Load only a batch of images from file paths with enhanced preprocessing"""
    batch_images = []
    batch_labels = []
    valid_indices = []

    for i, idx in enumerate(indices):
        try:
            img_path = file_paths[idx]
            img_array = cv2.imread(img_path)
            if img_array is not None:
                # Convert BGR to RGB
                img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
                img_array = cv2.resize(img_array, (img_size, img_size))

                # Enhanced preprocessing - histogram equalization for better contrast
                img_array = cv2.convertScaleAbs(img_array, alpha=1.1, beta=10)

                img_array = img_array.astype(np.float32) / 255.0  # Normalize
                batch_images.append(img_array)
                batch_labels.append(labels[idx])
                valid_indices.append(idx)
        except Exception as e:
            print(f"Error loading image {idx}: {e}")
            continue

    return np.array(batch_images), np.array(batch_labels), valid_indices

# Enhanced CNN model with Swish activation function
def create_enhanced_model():
    """
    Create enhanced CNN model with MobileNet base + custom layers
    Using Swish activation function instead of ReLU
    """
    from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Dense, Flatten
    from tensorflow.keras.regularizers import l2

    input_tensor = Input(shape=(224, 224, 3))

    # Base Model - MobileNet
    base_model = MobileNet(
        input_tensor=input_tensor,
        weights='imagenet',
        include_top=False  # Remove classification layers
    )

    # More aggressive fine-tuning - unfreeze more layers
    for layer in base_model.layers[:-30]:  # Freeze fewer layers (changed from -20 to -30)
        layer.trainable = False
    for layer in base_model.layers[-30:]:  # Unfreeze last 30 layers (changed from 20 to 30)
        layer.trainable = True

    # Add custom layers with Swish activation
    x = base_model.output

    # Custom Conv2D(512) + BatchNorm + MaxPool + Dropout with Swish
    x = Conv2D(512, (3, 3), activation='swish', padding='same', kernel_regularizer=l2(0.0005))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.2)(x)

    # Custom Conv2D(256) + BatchNorm + MaxPool + Dropout with Swish
    x = Conv2D(256, (3, 3), activation='swish', padding='same', kernel_regularizer=l2(0.0005))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.2)(x)

    # Custom Conv2D(128) + BatchNorm + Dropout with Swish
    x = Conv2D(128, (3, 3), activation='swish', padding='same', kernel_regularizer=l2(0.0005))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    # GlobalAveragePooling2D
    x = GlobalAveragePooling2D()(x)

    # Dense layers with Swish activation
    x = Dense(512, activation='swish', kernel_regularizer=l2(0.0005))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(256, activation='swish', kernel_regularizer=l2(0.0005))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(128, activation='swish', kernel_regularizer=l2(0.0005))(x)
    x = Dropout(0.3)(x)

    # Final classification layer
    predictions = Dense(4, activation='softmax')(x)  # 4 classes for brain tumor types

    # Create the complete model
    model = Model(inputs=base_model.input, outputs=predictions)

    print(f"Enhanced model created with Swish activation and {len(model.layers)} layers")
    print(f"Trainable parameters: {model.count_params()}")

    return model

# Create feature extractor from trained CNN
def create_feature_extractor(trained_model):
    """
    Create a feature extractor from the trained CNN model
    Extract features from the last dense layer before classification
    """
    # Get the layer before the final classification layer
    feature_layer = trained_model.layers[-2]  # Second to last layer (before softmax)

    # Create feature extractor model
    feature_extractor = Model(
        inputs=trained_model.input,
        outputs=feature_layer.output
    )

    print(f"Feature extractor created - extracting {feature_layer.output.shape[-1]} features")
    return feature_extractor

# Batch-wise feature extraction function
def extract_features_in_batches(feature_extractor, X_data, batch_size=32):
    """
    Extract features in batches to avoid memory issues
    """
    n_samples = len(X_data)
    n_batches = (n_samples + batch_size - 1) // batch_size

    features_list = []

    for i in range(n_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, n_samples)

        batch_data = X_data[start_idx:end_idx]

        # Extract features for this batch
        batch_features = feature_extractor.predict(batch_data, verbose=0)
        features_list.append(batch_features)

        # Force garbage collection to free memory
        gc.collect()

        # Print progress
        if (i + 1) % 10 == 0 or i == n_batches - 1:
            print(f"Processed batch {i+1}/{n_batches}")

    # Concatenate all features
    all_features = np.concatenate(features_list, axis=0)

    return all_features

# XGBoost parameter combinations for grid search
def get_xgboost_param_combinations():
    """
    Define different XGBoost parameter combinations to test
    """
    param_combinations = [
        # Combination 1: Balanced performance
        {
            'n_estimators': 100,
            'max_depth': 6,
            'learning_rate': 0.1,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'reg_alpha': 0.1,
            'reg_lambda': 1.0
        },
        # Combination 2: More conservative (prevent overfitting)
        {
            'n_estimators': 150,
            'max_depth': 4,
            'learning_rate': 0.05,
            'subsample': 0.7,
            'colsample_bytree': 0.7,
            'reg_alpha': 0.5,
            'reg_lambda': 2.0
        },
        # Combination 3: More aggressive (higher capacity)
        {
            'n_estimators': 200,
            'max_depth': 8,
            'learning_rate': 0.15,
            'subsample': 0.9,
            'colsample_bytree': 0.9,
            'reg_alpha': 0.01,
            'reg_lambda': 0.5
        },
        # Combination 4: High learning rate, shallow trees
        {
            'n_estimators': 80,
            'max_depth': 3,
            'learning_rate': 0.2,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'reg_alpha': 0.3,
            'reg_lambda': 1.5
        },
        # Combination 5: Low learning rate, deep trees
        {
            'n_estimators': 300,
            'max_depth': 10,
            'learning_rate': 0.03,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'reg_alpha': 0.2,
            'reg_lambda': 1.0
        },
        # Combination 6: Moderate settings with high regularization
        {
            'n_estimators': 120,
            'max_depth': 5,
            'learning_rate': 0.08,
            'subsample': 0.75,
            'colsample_bytree': 0.75,
            'reg_alpha': 1.0,
            'reg_lambda': 3.0
        },
        # Combination 7: Fast training setup
        {
            'n_estimators': 50,
            'max_depth': 4,
            'learning_rate': 0.3,
            'subsample': 0.9,
            'colsample_bytree': 0.9,
            'reg_alpha': 0.1,
            'reg_lambda': 0.5
        },
        # Combination 8: Comprehensive model
        {
            'n_estimators': 250,
            'max_depth': 7,
            'learning_rate': 0.06,
            'subsample': 0.85,
            'colsample_bytree': 0.85,
            'reg_alpha': 0.3,
            'reg_lambda': 1.2
        }
    ]

    return param_combinations

# Train XGBoost with parameter tuning
def train_xgboost_with_tuning(X_train_features, y_train, X_test_features, y_test):
    """
    Train XGBoost with different parameter combinations and find the best one
    """
    param_combinations = get_xgboost_param_combinations()
    best_accuracy = 0
    best_params = None
    best_model = None
    best_results = None

    print(f"Testing {len(param_combinations)} XGBoost parameter combinations...")
    print("="*80)

    for i, params in enumerate(param_combinations, 1):
        print(f"\nTesting Combination {i}/{len(param_combinations)}:")
        print(f"Parameters: {params}")

        # Train XGBoost with current parameters
        start_time = time.time()
        xgb_model = xgb.XGBClassifier(
            objective='multi:softprob',
            num_class=4,
            random_state=seed_value,
            n_jobs=-1,
            **params
        )

        xgb_model.fit(X_train_features, y_train)
        training_time = time.time() - start_time

        # Make predictions
        start_time = time.time()
        y_pred = xgb_model.predict(X_test_features)
        y_pred_proba = xgb_model.predict_proba(X_test_features)
        prediction_time = time.time() - start_time

        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
        mcc = matthews_corrcoef(y_test, y_pred)

        current_results = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'mcc': mcc,
            'training_time': training_time,
            'prediction_time': prediction_time,
            'params': params,
            'model': xgb_model,
            'y_pred': y_pred,
            'y_pred_proba': y_pred_proba
        }

        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-Score: {f1:.4f}")
        print(f"MCC: {mcc:.4f}")
        print(f"Training Time: {training_time:.2f}s")
        print(f"Prediction Time: {prediction_time:.2f}s")

        # Update best results if current is better
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = params
            best_model = xgb_model
            best_results = current_results
            print(f"*** NEW BEST ACCURACY: {accuracy:.4f} ***")

        print("-" * 60)

    print(f"\nBest XGBoost Configuration Found:")
    print(f"Best Accuracy: {best_accuracy:.4f}")
    print(f"Best Parameters: {best_params}")

    return best_model, best_results, best_params

# Enhanced training with CNN + XGBoost hybrid approach
def train_cnn_xgboost_hybrid(file_paths, file_labels, test_size=0.2, batch_size=16, epochs=30):
    """
    Train CNN model first, then use best weights as feature extractor for XGBoost
    """
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
    from tensorflow.keras.utils import to_categorical

    # Create enhanced model
    print("Creating enhanced CNN model with Swish activation...")
    cnn_model = create_enhanced_model()

    # Fine-tuned optimizer with slightly higher learning rate
    cnn_model.compile(
        optimizer=Adam(learning_rate=0.00015, beta_1=0.9, beta_2=0.999),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Split data into train and test
    train_idx, test_idx = train_test_split(
        range(len(file_paths)),
        test_size=test_size,
        random_state=seed_value,
        stratify=file_labels
    )

    print(f"Train samples: {len(train_idx)}, Test samples: {len(test_idx)}")

    # Load training images
    print("Loading training images...")
    start_time = time.time()
    X_train, y_train, valid_train_idx = load_image_batch(
        file_paths, file_labels, train_idx, IMG_SIZE
    )
    print(f"Loaded {len(X_train)} training images in {time.time() - start_time:.2f}s")

    # Load test images
    print("Loading test images...")
    start_time = time.time()
    X_test, y_test, valid_test_idx = load_image_batch(
        file_paths, file_labels, test_idx, IMG_SIZE
    )
    print(f"Loaded {len(X_test)} test images in {time.time() - start_time:.2f}s")

    if len(X_train) == 0 or len(X_test) == 0:
        print("Error: No valid images loaded!")
        return None

    # Convert labels to categorical (one-hot encoding) for CNN training
    y_train_cat = to_categorical(y_train, num_classes=4)
    y_test_cat = to_categorical(y_test, num_classes=4)

    # Light data augmentation
    train_datagen = ImageDataGenerator(
        rotation_range=5,
        width_shift_range=0.05,
        height_shift_range=0.05,
        horizontal_flip=True,
        zoom_range=0.05,
        fill_mode='nearest'
    )

    # Optimized callbacks
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=15,
            restore_best_weights=True,
            verbose=1,
            mode='max'
        ),
        ReduceLROnPlateau(
            monitor='val_accuracy',
            factor=0.5,
            patience=8,
            min_lr=0.00001,
            verbose=1,
            mode='max'
        ),
        ModelCheckpoint(
            'best_brain_tumor_model_swish.h5',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1,
            mode='max'
        )
    ]

    # Train the CNN model
    print("Starting CNN training with Swish activation...")
    start_time = time.time()

    train_datagen.fit(X_train)

    history = cnn_model.fit(
        train_datagen.flow(X_train, y_train_cat, batch_size=batch_size),
        steps_per_epoch=len(X_train) // batch_size,
        epochs=epochs,
        validation_data=(X_test, y_test_cat),
        callbacks=callbacks,
        verbose=1
    )

    cnn_training_time = time.time() - start_time
    print(f"CNN training completed in {cnn_training_time:.2f}s")

    # Load the best CNN model weights
    print("Loading best CNN model weights...")
    best_cnn_model = keras.models.load_model('best_brain_tumor_model_swish.h5')

    # Create feature extractor from the best CNN model
    print("Creating feature extractor from best CNN weights...")
    feature_extractor = create_feature_extractor(best_cnn_model)

    # Extract features for XGBoost training using batch processing
    print("Extracting features for XGBoost training...")
    start_time = time.time()

    # Use batch-wise feature extraction to avoid memory issues
    print("Extracting training features in batches...")
    X_train_features = extract_features_in_batches(feature_extractor, X_train, batch_size=32)

    # Force garbage collection
    gc.collect()

    print("Extracting test features in batches...")
    X_test_features = extract_features_in_batches(feature_extractor, X_test, batch_size=32)

    # Force garbage collection
    gc.collect()

    feature_extraction_time = time.time() - start_time
    print(f"Feature extraction completed in {feature_extraction_time:.2f}s")
    print(f"Feature shape: {X_train_features.shape}")

    # Standardize features for XGBoost
    print("Standardizing features for XGBoost...")
    scaler = StandardScaler()
    X_train_features_scaled = scaler.fit_transform(X_train_features)
    X_test_features_scaled = scaler.transform(X_test_features)

    # Train XGBoost with parameter tuning
    print("Training XGBoost with parameter tuning...")
    best_xgb_model, best_xgb_results, best_params = train_xgboost_with_tuning(
        X_train_features_scaled, y_train, X_test_features_scaled, y_test
    )

    # Get CNN accuracy from history
    train_accuracies = history.history['accuracy']
    val_accuracies = history.history['val_accuracy']
    highest_train_acc = max(train_accuracies)
    highest_val_acc = max(val_accuracies)

    # Results dictionary
    results = {
        'cnn_highest_train_acc': highest_train_acc,
        'cnn_highest_val_acc': highest_val_acc,
        'xgb_accuracy': best_xgb_results['accuracy'],
        'xgb_precision': best_xgb_results['precision'],
        'xgb_recall': best_xgb_results['recall'],
        'xgb_f1': best_xgb_results['f1'],
        'xgb_mcc': best_xgb_results['mcc'],
        'cnn_training_time': cnn_training_time,
        'xgb_training_time': best_xgb_results['training_time'],
        'feature_extraction_time': feature_extraction_time,
        'xgb_prediction_time': best_xgb_results['prediction_time'],
        'n_train_samples': len(X_train),
        'n_test_samples': len(X_test),
        'n_features': X_train_features.shape[1],
        'y_true': y_test,
        'y_pred': best_xgb_results['y_pred'],
        'y_pred_proba': best_xgb_results['y_pred_proba'],
        'cnn_model': best_cnn_model,
        'feature_extractor': feature_extractor,
        'xgb_model': best_xgb_model,
        'best_xgb_params': best_params,
        'scaler': scaler,
        'cnn_history': history
    }

    return results

# Display results without graphs
def display_results(results):
    """
    Display results from CNN + XGBoost hybrid model (text only)
    """
    print("\n" + "="*80)
    print("CNN TRAINING RESULTS (WITH SWISH ACTIVATION)")
    print("="*80)
    print(f"Highest CNN Training Accuracy: {results['cnn_highest_train_acc']:.4f}")
    print(f"Highest CNN Validation Accuracy: {results['cnn_highest_val_acc']:.4f}")
    print(f"CNN Training Time: {results['cnn_training_time']:.2f}s")

    print("\n" + "="*80)
    print("CNN + XGBOOST HYBRID CLASSIFICATION RESULTS")
    print("="*80)
    print(f"XGBoost Accuracy: {results['xgb_accuracy']:.4f}")
    print(f"XGBoost Precision: {results['xgb_precision']:.4f}")
    print(f"XGBoost Recall: {results['xgb_recall']:.4f}")
    print(f"XGBoost F1-Score: {results['xgb_f1']:.4f}")
    print(f"Matthews Correlation Coefficient: {results['xgb_mcc']:.4f}")

    print(f"\nDataset Information:")
    print(f"Training samples: {results['n_train_samples']}")
    print(f"Test samples: {results['n_test_samples']}")
    print(f"Features extracted: {results['n_features']}")

    print(f"\nTiming Information:")
    print(f"Feature extraction time: {results['feature_extraction_time']:.2f}s")
    print(f"XGBoost training time: {results['xgb_training_time']:.2f}s")
    print(f"XGBoost prediction time: {results['xgb_prediction_time']:.2f}s")
    print(f"Total training time: {results['cnn_training_time'] + results['xgb_training_time']:.2f}s")

    print(f"\nBest XGBoost Parameters:")
    for param, value in results['best_xgb_params'].items():
        print(f"  {param}: {value}")

    # Classification report
    print(f"\nDetailed Classification Report:")
    print(classification_report(results['y_true'], results['y_pred'], target_names=labels, digits=4))

    # Confusion Matrix (text format)
    print(f"\nConfusion Matrix:")
    cm = confusion_matrix(results['y_true'], results['y_pred'])
    print("Predicted ->")
    print(f"{'Actual':<12} {'Glioma':<8} {'Meningioma':<12} {'No Tumor':<10} {'Pituitary':<10}")
    print("-" * 60)
    for i, label in enumerate(labels):
        print(f"{label:<12} {cm[i][0]:<8} {cm[i][1]:<12} {cm[i][2]:<10} {cm[i][3]:<10}")

    print("\n" + "="*80)
    print("FINAL SUMMARY - CNN + XGBOOST HYBRID MODEL")
    print("="*80)
    print(f"Feature Extractor: Enhanced MobileNet CNN with Swish activation")
    print(f"Classifier: XGBoost (optimized parameters)")
    print(f"CNN Highest Training Accuracy: {results['cnn_highest_train_acc']:.4f}")
    print(f"CNN Highest Validation Accuracy: {results['cnn_highest_val_acc']:.4f}")
    print(f"XGBoost Final Test Accuracy: {results['xgb_accuracy']:.4f}")
    print(f"XGBoost Final Precision: {results['xgb_precision']:.4f}")
    print(f"XGBoost Final Recall: {results['xgb_recall']:.4f}")
    print(f"XGBoost Final F1-Score: {results['xgb_f1']:.4f}")
    print(f"Matthews Correlation Coefficient: {results['xgb_mcc']:.4f}")
    print(f"Total Training Time: {results['cnn_training_time'] + results['xgb_training_time']:.2f}s")
    print("="*80)

    return results

# Main function with CNN + XGBoost hybrid approach
def main_hybrid(test_size=0.2, epochs=30):
    """
    Main function to run CNN + XGBoost hybrid model for brain tumor classification
    """
    print("Creating file paths list...")
    file_paths, file_labels = create_file_paths_list()

    # Train CNN + XGBoost hybrid model
    results = train_cnn_xgboost_hybrid(
        file_paths, file_labels,
        test_size=test_size,
        batch_size=16,
        epochs=epochs
    )

    if results is None:
        print("Failed to train hybrid model!")
        return None

    # Display results
    display_results(results)

    return results

# Run the CNN + XGBoost hybrid model
if __name__ == "__main__":
    # Run CNN + XGBoost hybrid model with parameter tuning
    results = main_hybrid(test_size=0.2, epochs=30)

Mounted at /content/drive
Data extracted successfully!
Data path: /content/extracted_data/sartaj1
Creating file paths list...
Found 1621 images in glioma
Found 1645 images in meningioma
Found 2000 images in notumor
Found 1457 images in pituitary
Total files found: 6723
Class distribution: [1621 1645 2000 1457]
Creating enhanced CNN model with Swish activation...


/tmp/ipython-input-2-3249993399.py:136: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNet(


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Enhanced model created with Swish activation and 107 layers
Trainable parameters: 9660356
Train samples: 5378, Test samples: 1345
Loading training images...
Loaded 5378 training images in 10.64s
Loading test images...
Loaded 1345 test images in 3.40s
Starting CNN training with Swish activation...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30


...

336/336 ━━━━━━━━━━━━━━━━━━━━ 78s 191ms/step - accuracy: 0.9235 - loss: 1.1125 - val_accuracy: 0.9688 - val_loss: 0.9707 - learning_rate: 1.5000e-04
Epoch 10/30
  1/336 ━━━━━━━━━━━━━━━━━━━━ 14s 43ms/step - accuracy: 1.0000 - loss: 0.9680
Epoch 10: val_accuracy did not improve from 0.96877
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 1.0000 - loss: 0.9680 - val_accuracy: 0.9688 - val_loss: 0.9713 - learning_rate: 1.5000e-04
Epoch 11/30
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - accuracy: 0.9403 - loss: 1.0567
Epoch 11: val_accuracy did not improve from 0.96877
336/336 ━━━━━━━━━━━━━━━━━━━━ 65s 193ms/step - accuracy: 0.9403 - loss: 1.0566 - val_accuracy: 0.9643 - val_loss: 0.9597 - learning_rate: 1.5000e-04
Epoch 12/30
  1/336 ━━━━━━━━━━━━━━━━━━━━ 13s 39ms/step - accuracy: 0.9375 - loss: 0.9701
Epoch 12: val_accuracy did not improve from 0.96877
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9375 - loss: 0.9701 - val_accuracy: 0.9643 - val_loss: 0.9596 - learning_rate: 1.

336/336 ━━━━━━━━━━━━━━━━━━━━ 65s 194ms/step - accuracy: 0.9516 - loss: 1.0000 - val_accuracy: 0.9792 - val_loss: 0.9336 - learning_rate: 1.5000e-04
Epoch 14/30
  1/336 ━━━━━━━━━━━━━━━━━━━━ 19s 57ms/step - accuracy: 0.9375 - loss: 1.0418
Epoch 14: val_accuracy did not improve from 0.97918
336/336 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9375 - loss: 1.0418 - val_accuracy: 0.9792 - val_loss: 0.9331 - learning_rate: 1.5000e-04
Epoch 15/30
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.9591 - loss: 0.9740
Epoch 15: val_accuracy did not improve from 0.97918
336/336 ━━━━━━━━━━━━━━━━━━━━ 79s 191ms/step - accuracy: 0.9591 - loss: 0.9740 - val_accuracy: 0.9717 - val_loss: 0.9056 - learning_rate: 1.5000e-04
Epoch 16/30
  1/336 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 1.0000 - loss: 0.8555
Epoch 16: val_accuracy did not improve from 0.97918
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 1.0000 - loss: 0.8555 - val_accuracy: 0.9710 - val_loss: 0.9050 - learning_rate: 1.

336/336 ━━━━━━━━━━━━━━━━━━━━ 82s 197ms/step - accuracy: 0.9707 - loss: 0.9121 - val_accuracy: 0.9814 - val_loss: 0.8460 - learning_rate: 1.5000e-04
Epoch 20/30
  1/336 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 1.0000 - loss: 0.8121
Epoch 20: val_accuracy did not improve from 0.98141
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 1.0000 - loss: 0.8121 - val_accuracy: 0.9814 - val_loss: 0.8439 - learning_rate: 1.5000e-04
Epoch 21/30
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step - accuracy: 0.9692 - loss: 0.8856
Epoch 21: val_accuracy did not improve from 0.98141
336/336 ━━━━━━━━━━━━━━━━━━━━ 64s 191ms/step - accuracy: 0.9692 - loss: 0.8857 - val_accuracy: 0.9807 - val_loss: 0.8482 - learning_rate: 1.5000e-04
Epoch 22/30
  1/336 ━━━━━━━━━━━━━━━━━━━━ 12s 36ms/step - accuracy: 1.0000 - loss: 0.8133
Epoch 22: val_accuracy did not improve from 0.98141
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 1.0000 - loss: 0.8133 - val_accuracy: 0.9807 - val_loss: 0.8474 - learning_rate: 1.

336/336 ━━━━━━━━━━━━━━━━━━━━ 79s 194ms/step - accuracy: 0.9642 - loss: 0.8625 - val_accuracy: 0.9918 - val_loss: 0.7576 - learning_rate: 1.5000e-04
Epoch 28/30
  1/336 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step - accuracy: 1.0000 - loss: 0.7609
Epoch 28: val_accuracy did not improve from 0.99182
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 1.0000 - loss: 0.7609 - val_accuracy: 0.9911 - val_loss: 0.7581 - learning_rate: 1.5000e-04
Epoch 29/30
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - accuracy: 0.9594 - loss: 0.8674
Epoch 29: val_accuracy did not improve from 0.99182
336/336 ━━━━━━━━━━━━━━━━━━━━ 66s 196ms/step - accuracy: 0.9594 - loss: 0.8675 - val_accuracy: 0.9792 - val_loss: 0.7872 - learning_rate: 1.5000e-04
Epoch 30/30
  1/336 ━━━━━━━━━━━━━━━━━━━━ 13s 39ms/step - accuracy: 1.0000 - loss: 0.7993
Epoch 30: val_accuracy did not improve from 0.99182
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 1.0000 - loss: 0.7993 - val_accuracy: 0.9792 - val_loss: 0.7871 - learning_rate: 1.

Creating feature extractor from best CNN weights...
Feature extractor created - extracting 128 features
Extracting features for XGBoost training...
Extracting training features in batches...
Processed batch 10/169
Processed batch 20/169
Processed batch 30/169
Processed batch 40/169
Processed batch 50/169
Processed batch 60/169
Processed batch 70/169
Processed batch 80/169
Processed batch 90/169
Processed batch 100/169
Processed batch 110/169
Processed batch 120/169
Processed batch 130/169
Processed batch 140/169
Processed batch 150/169
Processed batch 160/169
Processed batch 169/169
Extracting test features in batches...
Processed batch 10/43
Processed batch 20/43
Processed batch 30/43
Processed batch 40/43
Processed batch 43/43
Feature extraction completed in 90.78s
Feature shape: (5378, 128)
Standardizing features for XGBoost...
Training XGBoost with parameter tuning...
Testing 8 XGBoost parameter combinations...

Testing Combination 1/8:
Parameters: {'n_estimators': 100, 'max_depth'